In [1]:
!git clone https://github.com/tiendatnguyen2266/Do-An-2
%cd Do-An-2
!pip -q install -r setup.txt

Cloning into 'Do-An-2'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 31 (delta 11), reused 25 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 102.46 KiB | 25.62 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/Do-An-2


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip -q install pyngrok

In [6]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from flask import Flask, request, render_template
from pyngrok import ngrok, conf
from google.colab import userdata

# Định nghĩa tham số
rand_seed = torch.manual_seed(42)  # Đặt seed ngẫu nhiên
NUM_INFERENCE_STEPS = 100          # Số bước suy diễn
GUIDANCE_SCALE = 0.75              # Độ nhạy của ảnh theo prompt
HEIGHT = 512
WIDTH = 512

# Đường dẫn LoRA
LORA_PATH = "/content/gdrive/MyDrive/SD-Data/Lora/ghexanh.safetensors"

# Danh sách model
model_list = ["nota-ai/bk-sdm-small",
              "CompVis/stable-diffusion-v1-4",
              "runwayml/stable-diffusion-v1-5",
              "prompthero/openjourney",
              "shuttleai/shuttle-3-diffusion",
              "stabilityai/stable-diffusion-2-1",
              "dreamlike-art/dreamlike-photoreal-2.0"]

def create_pipeline(model_name = model_list[6]):
    # Nếu máy có GPU CUDA
    if torch.cuda.is_available():
        print("Using GPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype = torch.float16,
            use_safetensors = True
        ).to("cuda")
    else:
        print("Using CPU")
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_name,
            torch_dtype=torch.float32,
            use_safetensors=True
        )

    # Tải LoRA
    try:
        print("Loading LoRA...")
        pipeline.load_lora_weights(LORA_PATH)
        print("LoRA loaded successfully.")
    except Exception as e:
        print(f"Error loading LoRA: {e}")

    # Cấu hình scheduler (tùy chọn)
    pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

    return pipeline

def text2img(prompt, pipeline):
    images = pipeline(
        prompt,
        guidance_scale=GUIDANCE_SCALE,
        num_inference_steps=NUM_INFERENCE_STEPS,
        generator=rand_seed,
        height=HEIGHT,
        width=WIDTH
    ).images
    return images[0]

# Khởi tạo Flask app
app = Flask(__name__)

# Định nghĩa tham số
IMAGE_PATH = "static/output.jpg"
conf.get_default().auth_token = userdata.get('ngrok_token')

public_url = ngrok.connect(8888).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Khởi tạo pipeline
pipeline = create_pipeline()

@app.route("/", methods=['GET', 'POST'])
def index():
    if request.method == "GET":
        return render_template("index.html")
    else:
        user_input = request.form["prompt"]
        print("Start gen....")
        im = text2img(user_input, pipeline)
        print("Finish gen....")
        im.save(IMAGE_PATH)
        return render_template("index.html", image_url=IMAGE_PATH)

if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0', port=8888, use_reloader=False)

 * ngrok tunnel "https://6165-34-126-177-243.ngrok-free.app" -> "http://127.0.0.1:5000/"
Using GPU


model_index.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

Start gen....


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 19:33:59] "POST / HTTP/1.1" 200 -


Finish gen....


INFO:werkzeug:127.0.0.1 - - [08/Dec/2024 19:34:00] "GET /static/output.jpg HTTP/1.1" 200 -
